In [ ]:
!pip install wandb
!pip install simpletransformers

     |████████████████████████████████| 2.1MB 5.9MB/s 
     |████████████████████████████████| 102kB 10.4MB/s 
     |████████████████████████████████| 133kB 17.3MB/s 
     |████████████████████████████████| 163kB 16.4MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=dc353f91b84de898050f79ae531b073fae2df55709bb723963db5a820e17ee15
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=a4d09750f1a70362e872c6b8d700ed8cf904ff99284dfc9f2e89e091515ca64a
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools
     |████████████████████████████████| 215kB 5.8MB/s 
     |████████████████████████████████| 2.2MB 7.7MB/s 
     |████████████████████████████████|

In [ ]:
import logging

import pandas as pd
import sklearn
import wandb

import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from simpletransformers.classification import ClassificationArgs, ClassificationModel


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
sweep_config = {
    "method": "random",  # bayes, grid, random
    "metric": {"name": "train_loss", "goal": "minimize"},
    "parameters": {"manual_seed": {"values": [20,40,60,80,100]},
                   "num_train_epochs": {"values": [2,5,8,10]}},
    "early_terminate": {"type": "hyperband", "min_iter": 6},
}

# layer_parameters = {f"layer_{i}-{i + 6}": {"min": 0.0, "max": 5e-5} for i in range(0, 24, 6)}


# sweep_config = {
#     "method": "bayes",
#     "metric": {"name": "train_loss", "goal": "minimize"},
#     "parameters": {
#         "num_train_epochs": {"min": 5, "max": 15},
#         "params_classifier.dense.weight": {"min": 0, "max": 1e-3},
#         "params_classifier.dense.bias": {"min": 0, "max": 1e-3},
#         "params_classifier.out_proj.weight": {"min": 0, "max": 1e-3},
#         "params_classifier.out_proj.bias": {"min": 0, "max": 1e-3},
#         **layer_parameters,
#     },
#     "early_terminate": {"type": "hyperband", "min_iter": 6},
# }

# "parameters": {"num_train_epochs": {"values": [2,5,8,10]}, "learning_rate": {"min": 4e-6, "max": 4e-5}},

sweep_id = wandb.sweep(sweep_config, project="Simple Sweep_3")

Create sweep with ID: yl6mc9ax
Sweep URL: https://wandb.ai/ddave94/Simple%20Sweep_3/sweeps/yl6mc9ax


In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

### Importing Data

In [ ]:
#Creating the dataframes
kiva_train = pd.read_csv('https://raw.githubusercontent.com/DDave94/Loan-Default-Prediction/main/kiva_train.csv')
kiva_test = pd.read_csv('https://raw.githubusercontent.com/DDave94/Loan-Default-Prediction/main/kiva_test.csv')

#Formatting kaggle dataframe to run in the predict function
kaggle_test = kiva_test['en_clean']
kaggle_test = kaggle_test.to_list()

kiva_train.head(10)

,loan_id,en_clean,defaulted
0,7779,She opened a colmado out of the side of her ho...,0
1,2777,(First Loan): Joffre continues to run his loc...,1
2,6007,"Dina Santana is the mother of two children, Ju...",0
3,76,"Rosemary is 50 years old, single, and has 6 ch...",1
4,4217,"Segundo has a shop where he sells animal feed,...",0
5,5077,"I am a single parent, mother of 4. I sell groc...",0
6,6033,Mariana Jose Serda owns a general store in Mag...,0
7,843,Mary is a mother of four children. One of the ...,1
8,4357,Wilson is seeking his second loan with Kiva an...,0
9,7007,Clara lives with her two children (ages: 20 an...,0


### Preprocessing

In [ ]:
pip install unidecode textstat

In [ ]:
import unicodedata
import unidecode
stop_words = set(stopwords.words('english'))

lemmer = WordNetLemmatizer()

def preprocess(x):
    # Lower case
    x = x.lower()
    
    # Remove punctuation
    # x = re.sub(r'[^\w\s]', '', x)

    # Remove numbers
    # x = re.sub(r'\d+', '', x)
    
    # Remove non-unicode
    x = unidecode.unidecode(x)
        
    # Remove stopwords and lemmatize
    # x = [lemmer.lemmatize(w) for w in x.split() if w not in stop_words]
    x = [lemmer.lemmatize(w) for w in x.split()]
    return ' '.join(x) 

kiva_train['en_clean'] = kiva_train['en_clean'].apply(preprocess)

### Creating Train/Test Data

In [ ]:
# Creating Training Data
X = kiva_train['en_clean']
y = kiva_train['defaulted']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

train_df = pd.concat([X_train,y_train], axis = 1)
eval_df = pd.concat([X_val,y_val],axis = 1)

In [ ]:
model_args = ClassificationArgs()
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.evaluate_during_training = False
model_args.use_multiprocessing = True
model_args.no_cache = True
model_args.save_model_every_epoch = False
model_args.wandb_project = "Simple Sweep_3"
model_args.train_batch_size = 32
model_args.eval_batch_size = 32
model_args.max_seq_length = 300
#model_args.num_train_epochs = 8
model_args.learning_rate = 4e-5

#Man seed 20, epochs 10


In [ ]:
def train():
    # Initialize a new wandb run
    wandb.init()

    # Create a TransformerModel
    model = ClassificationModel("xlnet", "xlnet-base-cased", use_cuda=True, args=model_args, sweep_config=wandb.config)

    # Train the model
    model.train_model(train_df)

    # Evaluate the model
    model.eval_model(eval_df)

    preds, outputs = model.predict(X_val.to_list())
    print(classification_report(y_val, preds))

    # Sync wandb
    wandb.join()


wandb.agent(sweep_id, train)

INFO:wandb.agents.pyagent:Starting sweep agent: entity=None, project=None, count=None
wandb: Sweep Agent: Waiting for job.
ERROR:wandb.sdk.internal.internal_api:500 response executing GraphQL.
ERROR:wandb.sdk.internal.internal_api:{"errors":[{"message":"Post \"http://anaconda.default.svc.cluster.local/search\": context deadline exceeded","path":["agentHeartbeat"]}],"data":{"agentHeartbeat":null}}
wandb: ERROR Error while calling W&B API: Post "http://anaconda.default.svc.cluster.local/search": context deadline exceeded (<Response [500]>)
ERROR:wandb.sdk.internal.internal_api:500 response executing GraphQL.
ERROR:wandb.sdk.internal.internal_api:{"errors":[{"message":"context deadline exceeded","path":["agentHeartbeat"]}],"data":{"agentHeartbeat":null}}
wandb: ERROR Error while calling W&B API: context deadline exceeded (<Response [500]>)
wandb: Job received.
wandb: Agent Starting Run: z95belum with config:
wandb: 	manual_seed: 100
wandb: 	num_train_epochs: 8
wandb: Currently logged in a

INFO:filelock:Lock 140441879463312 acquired on /root/.cache/huggingface/transformers/06bdb0f5882dbb833618c81c3b4c996a0c79422fa2c95ffea3827f92fc2dba6b.da982e2e596ec73828dbae86525a1870e513bd63aae5a2dc773ccc840ac5c346.lock


Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

INFO:filelock:Lock 140441879463312 released on /root/.cache/huggingface/transformers/06bdb0f5882dbb833618c81c3b4c996a0c79422fa2c95ffea3827f92fc2dba6b.da982e2e596ec73828dbae86525a1870e513bd63aae5a2dc773ccc840ac5c346.lock
INFO:filelock:Lock 140444350970512 acquired on /root/.cache/huggingface/transformers/9461853998373b0b2f8ef8011a13b62a2c5f540b2c535ef3ea46ed8a062b16a9.3e214f11a50e9e03eb47535b58522fc3cc11ac67c120a9450f6276de151af987.lock


Downloading:   0%|          | 0.00/467M [00:00<?, ?B/s]

...

Run l7fobwbm errored: RuntimeError('CUDA out of memory. Tried to allocate 94.00 MiB (GPU 0; 15.90 GiB total capacity; 14.87 GiB already allocated; 3.75 MiB free; 15.02 GiB reserved in total by PyTorch)')
wandb: ERROR Run l7fobwbm errored: RuntimeError('CUDA out of memory. Tried to allocate 94.00 MiB (GPU 0; 15.90 GiB total capacity; 14.87 GiB already allocated; 3.75 MiB free; 15.02 GiB reserved in total by PyTorch)')
wandb: Agent Starting Run: 0p60q2vu with config:
wandb: 	manual_seed: 80
wandb: 	num_train_epochs: 5


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Run 0p60q2vu errored: RuntimeError('CUDA out of memory. Tried to allocate 94.00 MiB (GPU 0; 15.90 GiB total capacity; 14.87 GiB already allocated; 3.75 MiB free; 15.02 GiB reserved in total by PyTorch)')
wandb: ERROR Run 0p60q2vu errored: RuntimeError('CUDA out of memory. Tried to allocate 94.00 MiB (GPU 0; 15.90 GiB total capacity; 14.87 GiB already allocated; 3.75 MiB free; 15.02 GiB reserved in total by PyTorch)')
wandb: Agent Starting Run: olh45ni3 with config:
wandb: 	manual_seed: 40
wandb: 	num_train_epochs: 8


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Problem finishing run
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/wandb/agents/pyagent.py", line 303, in _run_job
    self._function()
  File "<ipython-input-33-63bfa49bbb10>", line 9, in train
    model.train_model(train_df)
  File "/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py", line 404, in train_model
    self._move_model_to_device()
  File "/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py", line 1631, in _move_model_to_device
    self.model.to(self.device)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 673, in to
    return self._apply(convert)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 387, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 387, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.7/dist